# Introduction

The song “12 Days of Christmas”, written around 1780, tells the tale of many gifts a person receives in the days leading up to Christmas (link to lyrics: https://en.wikipedia.org/wiki/The_Twelve_Days_of_Christmas_(song)).

These gifts repeat and compound; on the first day, the narrator receives

A partridge in a pear tree.
On the twelfth day, they receive

Twelve Drummers Drumming

Eleven Pipers Piping

Ten Lords a Leaping

Nine Ladies Waiting

Eight Maids a Milking

Seven Swans a Swimming

Six Geese a Laying

Five Golden Rings

Four Calling Birds

Three French Hens

Two Turtle Doves

And a Partridge in a Pear Tree


This week, your task will be to write functions that automatically sing this very repetitive song.

# Data set
Run the code provided to load in a data set called xmas that contains the crucial information about the gifts in the song. We will use this data set to test out our functions as we work on them.

In [752]:
import pandas as pd
import re
from num2words import num2words
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

In [753]:
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN
5,6,sixth,goose,a-laying,NaN,NaN
6,7,seventh,swan,a-swimming,NaN,NaN
7,8,eighth,maid,a-milking,NaN,NaN
8,9,ninth,lady,dancing,NaN,NaN
9,10,tenth,lord,a-leaping,NaN,NaN


# Advice
- If you have some trouble getting started, I recommend writing a function that works in one case, and then trying to generalize. For example, in building my sing_day() function, I might first write a version called sing_third_day() that sings:

        ```
        On the third day of Christmas, my true love gave to me:
        three french hens,
        two turtle doves,
        and a patridge in a pear tree.
        ```

- Make smaller versions of the xmas data set (e.g., the first two days). Once you feel confident in your function code, use the smaller version of the data to test the functions you write, before you test them on the full data set.

- Don’t sweat the small stuff. There’s a lot you can do to polish up the way the song prints. However, the goal of this lab is to practice writing functions and using iteration. Don’t get bogged down in details like how the song displays, or small grammar rules (like commas!), until you’ve finished the main tasks. You will have a chance to do this on the Challenge for this week!

# Function 1: pluralize_gift()
The gifts are listed in singular: for example, on day five the narrator receives “five golden rings”, but the entry in the data set for the gift on day five simply says “ring”.

- Hint 1: The gifts on days six and nine have unusual pluralization. You may assume that in other data sets, there will be no additional special cases besides these types.

- Hint 2: The following small code snippets may be useful to you:
        
        obj_1 = "foot"
        obj_2 = "baby"
        obj_3 = "tree"
        
        obj_1.find("oo")
        obj_2[-1]
        obj_3.find("oo")
        
        obj_1.replace("oo", "ee")
        obj_2.replace("y", "ies")
        obj_3 + "s"

In [754]:
def pluralize_gift(gift):
    """
    Returns the plural of a noun based on general English pluralization rules.

    Parameters
    ----------
    gift: str
      A noun
    
    Return
    ------
    str
      Plural version of the noun
    """
    
    # If the gift ends with 'y' preceded by a consonant, replace 'y' with 'ies'
    if gift[-1] == "y" and gift[-2] not in "aeiou":
        return gift[:-1] + "ies"
    
    # If the noun ends with 's', 'x', 'z', 'ch', or 'sh', add 'es'
    elif re.search(r'(s|x|z|ch|sh)$', gift):
        return gift + "es"
    
    # Special case for irregular pluralization of words ending in 'oo' (e.g., goose to geese)
    elif re.search(r'oo', gift):
        return gift.replace("oo", "ee")
    
    # Default case: simply add 's' for regular plurals
    else:
        return gift + "s"

## Test Your Function
Try your function out on the smaller and then larger gift data set. Consider: is your function vectorized? If not, how would you run it on all the gifts in the column.

In [755]:
# Should work
pluralize_gift("goose")
# Will work if your function is vectorized! 
# pluralize_gift(xmas['Gift.Item'])
# Since my function is not vectorized, I would run it on all gifts in the column as such:
xmas['Gift.Item'].apply(pluralize_gift)

0     partridges
1          doves
2           hens
3          birds
4          rings
5          geese
6          swans
7          maids
8         ladies
9          lords
10        pipers
11      drummers
Name: Gift.Item, dtype: object

In [756]:
# Testing the function with examples
print(pluralize_gift("baby"))  # Outputs: babies
print(pluralize_gift("box"))   # Outputs: boxes
print(pluralize_gift("tree"))  # Outputs: trees
print(pluralize_gift("goose")) # Outputs: geese (based on 'oo' to 'ee' rule)

babies
boxes
trees
geese


# Function 2: make_phrase()
Write a function called make_phrase() that takes as input the necessary information, and returns a phrase. For example,

        make_phrase(num_word = "ten", 
                    item = "lords", 
                    verb = "a-leaping", 
                    adjective = "", 
                    location = "")
should return

"ten lords a-leaping"

In [757]:
def make_phrase(num, item, verb, adjective, location):
    """
    Constructs a phrase for the '12 Days of Christmas' based on input parameters.
    
    Parameters
    ----------
    num: int
      The numeric day (e.g., 1, 2, 3, ...)
    item: str
      The gift item (e.g., "lords")
    verb: str or pd.Series
      The action or verb (e.g., "a-leaping")
    adjective: str or pd.Series
      Any adjective for the item (e.g., "golden")
    location: str or pd.Series
      Location phrase (e.g., "in the meadow")
    
    Returns
    -------
    str
      A formatted string representing the phrase
    """

    # Step 1: Replace NAs with blank strings
    verb = "" if pd.isna(verb) else verb
    adjective = "" if pd.isna(adjective) else adjective
    location = "" if pd.isna(location) else location
    
    ## Step 2: If the day number is larger than 1, pluralize the gift item
    if num > 1:
        item = pluralize_gift(item)
    
    ## Step 3: Check if the gift item starts with a vowel
    vowels = "AEIOUaeiou"
    starts_with_vowel = item[0] in vowels
    
    ## Step 4: Handle the article for the first day
    if num == 1:
        day_phrase = "an" if starts_with_vowel else "a"
    else:
        day_phrase = num2words(num)  # Use the number word for other days
    
    ## Step 5: Construct the final phrase
    phrase = f"{day_phrase} {adjective} {item} {verb}"
    
    if location:
        phrase += f" {location}"
    
    return phrase.replace("  ", " ")

## Test Your Function
Make sure to try your function out on small examples and on the xmas data.

Then, use the function to make a new column of the xmas column called Full.Phrase containing the sentences for the new gift on that day.

In [758]:
make_phrase(num=10,
            item="lord",
            verb="a-leaping",
            adjective="",
            location="")

'ten lords a-leaping'

In [759]:
# Apply the make_phrase function using the number to word mapping
xmas['Full.Phrase'] = xmas.apply(lambda x: make_phrase(num = x['Day'], 
                                                    item = x['Gift.Item'], 
                                                    verb = x['Verb'], 
                                                    adjective = x['Adjective'], 
                                                    location = x['Location']), axis = 1)

In [760]:
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Full.Phrase
0,1,first,partridge,NaN,NaN,in a pear tree,a partridge in a pear tree
1,2,second,dove,NaN,turtle,NaN,two turtle doves
2,3,third,hen,NaN,french,NaN,three french hens
3,4,fourth,bird,NaN,calling,NaN,four calling birds
4,5,fifth,ring,NaN,golden,NaN,five golden rings
5,6,sixth,goose,a-laying,NaN,NaN,six geese a-laying
6,7,seventh,swan,a-swimming,NaN,NaN,seven swans a-swimming
7,8,eighth,maid,a-milking,NaN,NaN,eight maids a-milking
8,9,ninth,lady,dancing,NaN,NaN,nine ladies dancing
9,10,tenth,lord,a-leaping,NaN,NaN,ten lords a-leaping


# Function 3: sing_day()
Write a function called sing_day() that takes as input:

- A dataset (input as a dataframe)

- A number indicating which day to sing about (input as an integer)

- The name of a column in the dataset that contains the phrases for each day (input as an tidy name)

For example,

        sing_day(xmas, 2, Full.Phrase)
should return

        On the second day of Christmas, my true love sent to me:
        two turtle doves and
        a partridge in a pear tree.

In [761]:
def sing_day(dataset, num, phrase_col):
    """
    This function sings the '12 Days of Christmas' song for a specific day.
    
    Parameters
    ----------
    dataset : pd.DataFrame
        The dataset containing the song phrases for each day.
    num : int
        The day number to sing about.
    phrase_col : str
        The column name that contains the phrases for each day.
        
    Returns
    -------
    str
        The song lyrics for the specified day.
    """
    num_word = num2words(num, ordinal = True)
    intro = "On the " + num_word + " day of Christmas, my true love sent to me:"
    
    # Step 2: Sing the gift phrases
    gifts = []
    for i in range(num, 0, -1):  # gifts are sung in reverse order
        gift = str(dataset.iloc[i - 1][phrase_col]).strip()  # Use .iloc for index-based access
        if i == 1 and num > 1:
            gift = "and " + gift  # Add "and" before the last gift for days > 1
        gifts.append(gift)
    
    # Step 3: Add commas and a period
    # Join gifts with commas, add a period at the end
    song_lyrics = ",\n".join(gifts) + "."
    
    # Step 4: Return the complete verse
    verse = intro + "\n" + song_lyrics.strip()
    return verse

## Test your function
Use this code to show the function works:

In [762]:
print(sing_day(xmas, 3, "Full.Phrase"))

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.


# Use Your Functions!
Run appropriate code to output the lyrics for the entire 12 Days of Christmas song.

In [763]:
# Generate the phrase for each of the twelve days using a lambda function
verses = list(map(lambda day: sing_day(xmas, day, 'Full.Phrase'), range(1, 13)))

print("\n\n".join(verses))

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me:
two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fourth day of Christmas, my true love sent to me:
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fifth day of Christmas, my true love sent to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the sixth day of Christmas, my true love sent to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the seventh day of Christmas, my true love sent to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a 


Then, load the following dataset, and run your code again on this dataset instead to get a surprise song! (The column names and formats of xmas2 are the same as those for xmas.)

In [764]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")

In [765]:
# Apply the make_phrase function using the number to word mapping
xmas2['Full.Phrase'] = xmas2.apply(lambda x: make_phrase(num = x['Day'], 
                                                    item = x['Gift.Item'], 
                                                    verb = x['Verb'], 
                                                    adjective = x['Adjective'], 
                                                    location = x['Location']), axis = 1)

In [766]:
xmas2

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Full.Phrase
0,1,first,email,NaN,NaN,from Cal Poly,an email from Cal Poly
1,2,second,point,NaN,meal,NaN,two meal points
2,3,third,pen,NaN,lost,NaN,three lost pens
3,4,fourth,review,NaN,course,NaN,four course reviews
4,5,fifth,exam,NaN,practice,NaN,five practice exams
5,6,sixth,grader,grading,NaN,NaN,six graders grading
6,7,seventh,senior,stressing,NaN,NaN,seven seniors stressing
7,8,eighth,mom,a-calling,NaN,NaN,eight moms a-calling
8,9,ninth,party,bumping,NaN,NaN,nine parties bumping
9,10,tenth,load,of laundry,NaN,NaN,ten loads of laundry


In [767]:
# Generate the phrase for each of the twelve days using a lambda function
verses2 = list(map(lambda day: sing_day(xmas2, day, 'Full.Phrase'), range(1, 13)))

print("\n\n".join(verses2))

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.

On the second day of Christmas, my true love sent to me:
two meal points,
and an email from Cal Poly.

On the third day of Christmas, my true love sent to me:
three lost pens,
two meal points,
and an email from Cal Poly.

On the fourth day of Christmas, my true love sent to me:
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the fifth day of Christmas, my true love sent to me:
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the sixth day of Christmas, my true love sent to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the seventh day of Christmas, my true love sent to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On t